# Статус

- рубрика 4. Список статей

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [36]:

u_l = ['4/', '40/', '3/']
#u_l = ['40/', '3/']

url_basic = 'https://www.kommersant.ru/archive/rubric/'
res = pd.DataFrame()
t0 = time.time()

for u in u_l:
    
    url_ = url_basic + u
    
    for y in range(2016,2020):
        for m in range(1,13):
            for d in range(1,32):

                    st_m = str(m)
                    if len(st_m) == 1: st_m = '0'+st_m

                    st_d = str(d)
                    if len(st_d) == 1: st_d = '0'+st_d

                    st = str(y) + '-' + st_m + '-' + st_d

                    url = url_ + st

                    try: 
                        page = requests.get(url)
                    except:
                        r = {'step':st,'rubric':u, 'error':'downloading error', 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))
                        continue

                    try:
                        p = BeautifulSoup(page.content, "html.parser")
                        url_list = p.find_all('article')
                    except:
                        r = {'step':st,'rubric':u, 'error':'parsing error', 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))


                    for x in url_list:
                        r = {'step':st,'rubric':u, 'url':x.find('a').get('href'), 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))

                    print(st, url_, end = '\r')



    #res.to_excel('res_40_2018.xlsx')

In [38]:
res.to_excel('texts/kommersant_r_2.xlsx', index = False)

In [41]:
res = pd.read_excel('texts/kommersant_r_1.xlsx').append(res)

In [42]:
def find_doc(x):
    return x.find('/doc/') != -1

res['flag'] = res['url'].apply(find_doc)
res[res['flag']]['rubric'].value_counts()

4/     76516
40/    69869
3/     69401
Name: rubric, dtype: int64

In [45]:
res = res[res['flag']]

In [48]:
res = res.drop_duplicates(['url'])

In [50]:
res.to_excel('texts/kommersant_r_total.xlsx', index = False)

In [2]:
from tqdm import tqdm

In [3]:
res = pd.read_excel('texts/kommersant_r_total.xlsx')

In [5]:
url_list = res.copy()

In [6]:


url_ = 'https://www.kommersant.ru'
#res = pd.read_excel('texts.xlsx')
res = pd.DataFrame()

urls = url_list['url'].tolist()



for x_ in tqdm(range(29811, len(urls))):
    
    x = urls[x_]
    url = url_ + x

    try:
        page = requests.get(url).content
    except:
        r = {'url': x, "error": "downloading error"}
        res = res.append(pd.DataFrame(r, index=[len(res)]))
        continue

    p = BeautifulSoup(page, "html.parser")
    p_ = p.find_all('p', {'class': "b-article__text"})
    st = ''
    for t in p_: st = st + t.text + ' '

    try:
        article_name = p.find_all('h2', {'class': "article_name"})[0].text
    except:
        try:
            article_name = p.find_all('h1', {'class': "article_name"})[0].text
        except:
            article_name = None

    try:
        article_subheader = p.find_all('h1', {'class': "article_subheader"})[0].text
    except:
        article_subheader = None

    try:
        authors = p.find_all('p', {'class': "b-article__text document_authors"})[0].text
    except:
        authors = None

    r = {'url': x, "text": st, "authors": authors, "article_header": article_name,
         "artical_subheader": article_subheader}
    res = res.append(pd.DataFrame(r, index=[len(res)]))


100%|██████████| 3878/3878 [1:03:52<00:00,  2.88it/s]


In [59]:
res.to_excel('texts/kommersant.xlsx', index = False)

In [57]:
res.to_excel('texts_1.xlsx')